In [1]:
import pandas as pd
import numpy as np
from datetime import date, time
from IPython.display import display
pd.options.display.max_columns = None

C:\Users\chris\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# create dict + function for error messages

error_msg = {}

def errormessage(summary, key, message):
    if key in summary.keys():
        error_msg[str(key)].append(str(message))
    else:
        error_msg[str(key)] = [str(message)]
        
print(error_msg)

{}


In [3]:
# name item to check

hospital = 'mri'
hosp_ts = hospital+'_ts.xlsx'
hosp_p = hospital+'_p.xlsx'

In [16]:
# load time series

ts = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\"+hosp_ts, sheet_name=None)

#rename sheets
try:
    ts['Anonymised Data'] = ts['Sheet1']
except:
    pass

try:
    ts['Anonymised Data'] = ts['Sheet 1']
except:
    pass

# load patient data and clean

p = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\"+hosp_p, sheet_name=None)

try:
    patient = pd.DataFrame(data=p['Sheet1'])
except:
    print('The sheet isnt called Sheet1, rename this')

# rename columns
patient.columns = patient.iloc[2]

# delete top rows with titles
patient = patient.iloc[3:]
patient.columns.name = None

# rename index
patient = patient.set_index(patient['sid_number'])
patient.index.name = None

# remove hospital numbers if present
if patient.columns[0] == 'hospital_number':
    patient = patient.iloc[:, 1:]
    
# delete all data below last patient number
sid_count = 0
for SID in patient['sid_number']:
    if not pd.isnull(SID):
        sid_count += 1
patient = patient.iloc[:sid_count]

# fill empty cells
patient.fillna(value=0, inplace=True)

# combine scheduled date and time columns from old version if this was used
if patient.columns[28] == 'scheduled_date':
    merged = []
    for row in patient.iterrows():
        row_data = row[1]
        time_merge = pd.Timestamp.combine(row_data[28], row_data[29])
        merged.append(time_merge)
    merged_series = pd.DataFrame(merged)
    merged_series = merged_series.set_index(patient['sid_number'])
    patient['scheduled_start'] = merged_series
    patient = patient.drop('scheduled_date', axis=1)
    
print(patient.info())

<class 'pandas.core.frame.DataFrame'>
Index: 224 entries, R0A02122 to R0A02121
Data columns (total 63 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sid_number             224 non-null    object 
 1   age                    224 non-null    int64  
 2   gender_male            224 non-null    int64  
 3   gender_female          224 non-null    int64  
 4   gender_non_binary      224 non-null    int64  
 5   asian                  224 non-null    int64  
 6   black                  224 non-null    int64  
 7   mixed                  224 non-null    int64  
 8   white                  224 non-null    int64  
 9   other                  224 non-null    int64  
 10  cardiac                224 non-null    int64  
 11  kidney                 224 non-null    int64  
 12  liver                  224 non-null    int64  
 13  neuro                  224 non-null    int64  
 14  pulm                   224 non-null    int64  
 15 

In [5]:
# check for mismatch in time series

time_series = pd.DataFrame(data=ts['Anonymised Data'])
mismatch = time_series.iloc[94, 5:18]
mis1 = mismatch.sum(axis=0)

if mis1 > 0:
    errormessage(error_msg, 'Input Data', 'There is an issue with the format of the Input Data tab of the Time-Series CRF. Usually this issue arises from extra spaces or commas in the Input Data tab causing a mismatch in the way hospital IDs are anonymised.')

In [6]:
# strip time series dataframe of counts for interaction checking

time_series.columns = time_series.iloc[0]
time_series = time_series.iloc[1:92, 18:31]
time_series

,otd_cancel,otd_cancel_nobed,ref_lvl1,plan_adm_lvl1,adm_lvl1,ref_lvl2,plan_adm_lvl2,adm_lvl2,adm_7d_lvl2,ref_lvl3,plan_adm_lvl3,adm_lvl3,adm_7d_lvl3
1,"R0A021, R0A022, R0A023, R0A024, R0A025",NaN,"R0A026, R0A027","R0A026, R0A027","R0A026, R0A027",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R0A028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"R0A029, R0A0210",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"R0A0211, R0A0212, R0A0213",NaN,"R0A0214, R0A0215, R0A0217, R0A0218","R0A0214, R0A0215, R0A0217, R0A0218","R0A0214, R0A0215, R0A0217, R0A0218",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R0A0219,NaN,R0A0220,R0A0220,R0A0220,"R0A0221, R0A0222, R0A0223","R0A0221, R0A0222, R0A0223","R0A0221, R0A0222, R0A0223","R0A0221, R0A0222, R0A0223",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,"R0A02389, R0A02390, R0A02391",NaN,"R0A02392, R0A02393","R0A02392, R0A02393","R0A02392, R0A02393, R0A02354","R0A02394, R0A02354",R0A02394,R0A02394,"R0A02394, R0A02392",NaN,NaN,NaN,NaN
89,"R0A02395, R0A02396",NaN,R0A02397,R0A02397,R0A02397,"R0A02398, R0A02399",R0A02398,R0A02398,R0A02398,NaN,NaN,R0A02399,R0A02399
90,"R0A02400, R0A02401, R0A02402, R0A02403",NaN,R0A02404,R0A02404,R0A02404,R0A02405,R0A02405,R0A02405,"R0A02405, R0A02407","R0A02407, R0A02406","R0A02407, R0A02406","R0A02407, R0A02406","R0A02407, R0A02406"


In [7]:
# create list of hospital numbers in each cell

def listify(cell):
    if pd.isna(cell):
        return []
    else:
        cell = str(cell)
        cell = cell.replace(" ", "")
        listy = list(cell.split(","))
        return listy

time_series = time_series.applymap(listify)
time_series

,otd_cancel,otd_cancel_nobed,ref_lvl1,plan_adm_lvl1,adm_lvl1,ref_lvl2,plan_adm_lvl2,adm_lvl2,adm_7d_lvl2,ref_lvl3,plan_adm_lvl3,adm_lvl3,adm_7d_lvl3
1,"[R0A021, R0A022, R0A023, R0A024, R0A025]",[],"[R0A026, R0A027]","[R0A026, R0A027]","[R0A026, R0A027]",[],[],[],[],[],[],[],[]
2,[R0A028],[],[],[],[],[],[],[],[],[],[],[],[]
3,"[R0A029, R0A0210]",[],[],[],[],[],[],[],[],[],[],[],[]
4,"[R0A0211, R0A0212, R0A0213]",[],"[R0A0214, R0A0215, R0A0217, R0A0218]","[R0A0214, R0A0215, R0A0217, R0A0218]","[R0A0214, R0A0215, R0A0217, R0A0218]",[],[],[],[],[],[],[],[]
5,[R0A0219],[],[R0A0220],[R0A0220],[R0A0220],"[R0A0221, R0A0222, R0A0223]","[R0A0221, R0A0222, R0A0223]","[R0A0221, R0A0222, R0A0223]","[R0A0221, R0A0222, R0A0223]",[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,[],[],[],[],[],[],[],[],[],[],[],[],[]
88,"[R0A02389, R0A02390, R0A02391]",[],"[R0A02392, R0A02393]","[R0A02392, R0A02393]","[R0A02392, R0A02393, R0A02354]","[R0A02394, R0A02354]",[R0A02394],[R0A02394],"[R0A02394, R0A02392]",[],[],[],[]
89,"[R0A02395, R0A02396]",[],[R0A02397],[R0A02397],[R0A02397],"[R0A02398, R0A02399]",[R0A02398],[R0A02398],[R0A02398],[],[],[R0A02399],[R0A02399]
90,"[R0A02400, R0A02401, R0A02402, R0A02403]",[],[R0A02404],[R0A02404],[R0A02404],[R0A02405],[R0A02405],[R0A02405],"[R0A02405, R0A02407]","[R0A02407, R0A02406]","[R0A02407, R0A02406]","[R0A02407, R0A02406]","[R0A02407, R0A02406]"


In [8]:
# check if site code has been added to hospital numbers

for row in time_series.iterrows():
    row_data = row[1]
    for item in row_data:
        if not item:
            pass
        else:
            if item[0].isdigit():
                errormessage(error_msg, 'Hospital Details', 'No hospital code has been added on the Hospital Details tab, please add one so that full study IDs can be generated. Once this is done, please ensure that the SIDs on the EhC referrals tab of the Time Series match up with the SIDs in the Patient CRF')
                break
            break
        break
    break

In [9]:
# iterate through rows of time series to conduct sense checks
for row in time_series.iterrows():
    row_data = row[1]
    
    # check if all cancellations due to a lack of ECB are in cancellations and ECB referrals

    # check if cancellations due to a lack of ECB is empty
    if row_data[1] != []:
        for SID in row_data[1]:
            
            # cancellations
            
            # check if cancellation field is empty or if SID appears in cancellations on that date
            if row_data[0] == [] or SID not in row_data[0]:
                errormessage(error_msg, 'The following list of patients are in the list of cancellations due to lack of ECB, but not in total cancellations for that day, please check this and amend on the Input Data tab.', SID)
                
            # referrals to each level
            
            # check if all referral fields are empty on that date
            if (type(row_data[2]) == float and type(row_data[5]) == float and type(row_data[9]) == float):
                errormessage(error_msg, 'The following list of patients are in the list of cancellations due to lack of ECB, but not in referrals to enhanced care, please check this and amend on the Input Data tab.', SID)
            
            # check if SID appears in any of the referral fields on that date
            else:
                if SID not in row_data[2] and SID not in row_data[5] and SID not in row_data[9]:
                    errormessage(error_msg, 'The following list of patients are in the list of cancellations due to lack of ECB, but not in referrals to enhanced care, please check this and amend on the Input Data tab.', SID)

    # check that cancellations dont end up admitted that day
    
    # check if cancellations is empty
    if row_data[0] != []:
        for SID in row_data[0]:
            
            # check if SID appears in any of the admission fields on that date
            if SID in row_data[4] or SID in row_data[7] or SID in row_data[11]:
                    errormessage(error_msg, 'The following list of patients are in the list of cancellations, but are also admitted the same day to a level of enhanced care, please check if this is correct and amend on the Input Data tab.', SID)
        
    # check that planned admissions are also in referrals, admissions and admissions within 7 days
    
    # level 1
    
    # check if planned admissions to level 1 is empty
    if row_data[3] != []:
        for SID in row_data[3]:
            
            # check if SID appears in the referral field for level 1
            if SID not in row_data[2]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 1, but not in referrals to level 1, please add them to the referrals for level 1 or remove them from planned admissions on the Input Data tab.', SID)
            if SID not in row_data[4]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 1, but not in admissions to level 1 for that day, please add them to the admissions for level 1 or remove them from planned admissions on the Input Data tab.', SID)
    
    # level 2
    
    # check if planned admissions to level 2 is empty
    if row_data[6] != []:
        for SID in row_data[6]:
            
            # check if SID appears in the referral field for level 2
            if SID not in row_data[5]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 2, but not in referrals to level 2, please add them to the referrals for level 2 or remove them from planned admissions on the Input Data tab.', SID)
            if SID not in row_data[7]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 2, but not in admissions to level 2 for that day, please add them to the admissions for level 2 or remove them from planned admissions on the Input Data tab.', SID)
            if SID not in row_data[8]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 2, but not in admissions to level 2 within 7 days, please add them to the admissions for level 2 within 7 days [all admissions on day 1 also should be in the within 7 days column] or remove them from planned admissions on the Input Data tab.', SID)
    
    # level 3
    
    # check if planned admissions to level 3 is empty
    if row_data[10] != []:
        for SID in row_data[10]:
            
            # check if SID appears in the referral field for level 3
            if SID not in row_data[9]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 3, but not in referrals to level 3, please add them to the referrals for level 3 or remove them from planned admissions on the Input Data tab.', SID)
            if SID not in row_data[11]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 3, but not in admissions to level 3 for that day, please add them to the admissions for level 3 or remove them from planned admissions on the Input Data tab.', SID)
            if SID not in row_data[12]:
                    errormessage(error_msg, 'The following list of patients are in the list of planned admissions to level 3, but not in admissions to level 3 within 7 days, please add them to the admissions for level 3 within 7 days [all admissions on day 1 also should be in the within 7 days column] or remove them from planned admissions on the Input Data tab.', SID)
     
    # check that admissions immediately after surgery appear in admissions within 7 days
    # and check if admissions match up with correct predicted level of enhanced care on patient CRF
    
    #level 1
    
    # check if admissions to level 1 is empty
    if row_data[4] != []:
        for SID in row_data[4]:
            
            # check if actual level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-5] != 1:
                    errormessage(error_msg, 'The following list of patients are in the list of admissions to level 1, but their actual post-op level of care in the patient CRF is different from this, please check this and amend as necessary.', SID)
            except:
                pass
    
    # level 2
    
    # check if admissions to level 2 is empty
    if row_data[7] != []:
        for SID in row_data[7]:
            
            # check if SID appears in admissions within 7 days for level 2
            if SID not in row_data[8]:
                    errormessage(error_msg, 'The following list of patients are in the list of admissions to level 2 immediately post-op, but not admissions within 7 days to level 2, please add them to admissions to level 2 within 7 days on the Input Data tab.', SID)

            # check if actual level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-5] != 2:
                    errormessage(error_msg, 'The following list of patients are in the list of admissions to level 2, but their actual post-op level of care in the patient CRF is different from this, please check this and amend as necessary.', SID)
            except:
                pass
            
    # level 3
    
    # check if admissions to level 3 is empty
    if row_data[11] != []:
        for SID in row_data[11]:
            
            # check if SID appears in admissions within 7 days for level 3
            if SID not in row_data[12]:
                    errormessage(error_msg, 'The following list of patients are in the list of admissions to level 3 immediately post-op, but not admissions within 7 days to level 3, please add them to admissions to level 3 within 7 days on the Input Data tab.', SID)
            
            # check if actual level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-5] != 3:
                    errormessage(error_msg, 'The following list of patients are in the list of admissions to level 3, but their actual post-op level of care in the patient CRF is different from this, please check this and amend as necessary.', SID)
            except:
                pass
    
    
    # check if all referrals are only made one per day and end up in either admissions, cancellations or ward stepdowns
    # and check if referrals match up with correct predicted level of enhanced care on patient CRF
    
    # level 1
    
    # check if referrals to level 1 is empty
    if row_data[2] != []:
        for SID in row_data[2]:
            
            # check if SID appears in any of the other referral fields on that date
            if SID in row_data[5] or SID in row_data[9]:
                    errormessage(error_msg, 'The following list of patients are in the list of referrals to level 1, but are also referred to another level of enhanced care that day, please check this and amend as necessary.', SID)
            
            # check if SID appears in any of the admission fields on that date or in cancellations
            if SID not in row_data[4] and SID not in row_data[7] and SID not in row_data[11] and SID not in row_data[0]:
                try:
                    if patient.loc[SID][-5] > 0:
                        errormessage(error_msg, 'The following list of patients are in the list of referrals to level 1, but are not admitted to any level of enhanced care, are not listed as cancelled, and are not listed as a ward stepdown in the Patient CRF, please check this and amend as necessary.', SID)
                except:
                    pass
                
            # check if predicted level of enhanced care is level 1 in patient CRF
            try:
                if patient.loc[SID][-6] != 1:
                    errormessage(error_msg, 'The following list of patients are in the list of referrals to level 1, but their predicted level of care in the patient CRF is different from this, please check this and amend as necessary.', SID)
            except:
                pass
    
    # level 2
    
    # check if referrals to level 2 is empty
    if row_data[5] != []:
        for SID in row_data[5]:
            
            # check if SID appears in any of the other referral fields on that date
            if SID in row_data[2] or SID in row_data[9]:
                    errormessage(error_msg, 'The following list of patients are in the list of referrals to level 2, but are also referred to another level of enhanced care that day, please check this and amend as necessary.', SID)
            
            # check if SID appears in any of the admission fields on that date or in cancellations
            if SID not in row_data[4] and SID not in row_data[7] and SID not in row_data[11] and SID not in row_data[0]:
                try:
                    if patient.loc[SID][-5] > 0:
                        errormessage(error_msg, 'The following list of patients are in the list of referrals to level 2, but iare not admitted to any level of enhanced care, are not listed as cancelled, and are not listed as a ward stepdown in the Patient CRF, please check this and amend as necessary.', SID)
                except:
                    pass
                
            # check if predicted level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-6] != 2:
                    errormessage(error_msg, 'The following list of patients are in the list of referrals to level 2, but their predicted level of care in the patient CRF is different from this, please check this and amend as necessary.', SID)
            except:
                pass
            
    # level 3
    
    # check if referrals to level 3 is empty
    if row_data[9] != []:
        for SID in row_data[9]:
            
            # check if SID appears in any of the other referral fields on that date
            if SID in row_data[2] or SID in row_data[5]:
                    errormessage(error_msg, 'The following list of patients are in the list of referrals to level 3, but are also referred to another level of enhanced care that day, please check this and amend as necessary.', SID)
            
            # check if SID appears in any of the admission fields on that date or in cancellations
            if SID not in row_data[4] and SID not in row_data[7] and SID not in row_data[11] and SID not in row_data[0]:
                try:
                    if patient.loc[SID][-5] > 0:
                        errormessage(error_msg, 'The following list of patients are in the list of referrals to level 3, but are not admitted to any level of enhanced care, are not listed as cancelled, and are not listed as a ward stepdown in the Patient CRF, please check this and amend as necessary.', SID)
                except:
                    pass
                
            # check if predicted level of enhanced care is correct in patient CRF
            try:
                if patient.loc[SID][-6] != 3:
                    errormessage(error_msg, 'The following list of patients are in the list of referrals to level 3, but their predicted level of care in the patient CRF is different from this, please check this and amend as necessary.', SID)        
            except:
                pass

In [10]:
# check if all referrals to ehanced care turn up in the patient CRF

EhC_refs = []

for cell in time_series['ref_lvl1']:
    EhC_refs = EhC_refs + cell
    
for cell in time_series['ref_lvl2']:
    EhC_refs = EhC_refs + cell
    
for cell in time_series['ref_lvl3']:
    EhC_refs = EhC_refs + cell
    
missing_SIDs = []
for SID in EhC_refs:
    if SID not in patient['sid_number']:
        missing_SIDs += [SID]
        
print(EhC_refs)
print(len(EhC_refs))
        
if len(missing_SIDs):
    errormessage(error_msg, 'The following list of patients are identified as referrals to enhanced care on the time series but do not appear in the Patient CRF, please add these.', missing_SIDs)
        
# check if all patients in patient CRF are in referrals on time series

missing_SIDs2 = []
for SID in patient['sid_number']:
    if SID not in EhC_refs:
        missing_SIDs2 += [SID]
        
if len(missing_SIDs2):
    errormessage(error_msg, 'The following list of patients are identified as referrals to enhanced care on the patient CRF but do not appear in the referrals columns in the time series, please add these.', missing_SIDs2)

['R0A026', 'R0A027', 'R0A0214', 'R0A0215', 'R0A0217', 'R0A0218', 'R0A0220', 'R0A0229', 'R0A0233', 'R0A0239', 'R0A0245', 'R0A0246', 'R0A0252', 'R0A0253', 'R0A0254', 'R0A0259', 'R0A0262', 'R0A0263', 'R0A0264', 'R0A0265', 'R0A0271', 'R0A0272', 'R0A0273', 'R0A0277', 'R0A0278', 'R0A0279', 'R0A0280', 'R0A0281', 'R0A0283', 'R0A0284', 'R0A0285', 'R0A0290', 'R0A0291', 'R0A0293', 'R0A0294', 'R0A0299', 'R0A02100', 'R0A02101', 'R0A02102', 'R0A02109', 'R0A02110', 'R0A02111', 'R0A02112', 'R0A02114', 'R0A02119', 'R0A02120', 'R0A0259', 'R0A02121', 'R0A02128', 'R0A02134', 'R0A02135', 'R0A02136', 'R0A02138', 'R0A02141', 'R0A02147', 'R0A02148', 'R0A02149', 'R0A02150', 'R0A02163', 'R0A02168', 'R0A02173', 'R0A02174', 'R0A02175', 'R0A02176', 'R0A02183', 'R0A02184', 'R0A02185', 'R0A02187', 'R0A02191', 'R0A02201', 'R0A02202', 'R0A02203', 'R0A02204', 'R0A02212', 'R0A02219', 'R0A02220', 'R0A02221', 'R0A02222', 'R0A02223', 'R0A02228', 'R0A02241', 'R0A02246', 'R0A02247', 'R0A02248', 'R0A02251', 'R0A02252', 'R0A02

In [11]:
# check if there are any double instances in the patient CRF list

SID_list = list(patient['sid_number'])
SID_unique = set(SID_list)
if len(SID_list) != len(SID_unique):
    seen = set()
    duplicates = []
    for SID in SID_list:
        if SID in seen:
            duplicates.append(SID)
        else:
            seen.add(SID)
    errormessage(error_msg, 'The following SIDs are duplicated in the patient CRF, which may be correct if they had two admissions during the study period, please check these.', duplicates)

In [12]:
# iterate through rows of patient CRF to conduct sense checks
for row in patient.iterrows():
    row_data = row[1]
    
    # check if age is >18
    if row_data[1] < 18:
        SID = row_data[0]
        errormessage(error_msg, 'The following list of patients have an age listed that is <18yrs, please check this and amend as necessary.', SID)
    
    # check if only 1 gender field flagged
    if row_data[2] + row_data[3] + row_data[4] != 1:
        SID = row_data[0]
        errormessage(error_msg, 'The following list of patients have an error in their gender denomination (either more than 1 selected or none at all), please check this and amend as necessary.', SID)
    
    # check if only 1 ethnicity field flagged
    if row_data[5] + row_data[6] + row_data[7] + row_data[8] + row_data[9] != 1:
        SID = row_data[0]
        errormessage(error_msg, 'The following list of patients have an error in their ethnicity denomination (either more than 1 selected or none at all), please check this and amend as necessary.', SID)
 
    # check if all those flagged as cancellations appear in cancellation column of time series and have a reason for cancellation
    
    # check if cancellation is flagged in patient CRF
    if row_data[31] == 1:
        
        # check if this SID appears in the cancellation field in the time series
        SID = row_data[0].replace(" ", "")
        count = 0
        for cell in time_series['otd_cancel']:
            if SID in cell:
                count += 1
                break
        if count == 0:
            errormessage(error_msg, 'The following list of patients are flagged as a cancellation in the patient CRF but do not appear in the time series as a cancellation on any day, please check this and amend as necessary.', SID)
        
        # check if reason a is flagged
        if row_data[42] != 1 and row_data[43] != 1:
            errormessage(error_msg, 'The following list of patients are flagged as a cancellation in the patient CRF but do not have a reason listed, either they were cancelled due to a lack of enhanced care bed OR this was for another reason, please flag one of these parameters for each.', SID)
        
    # check if all those flagged as cancellations due to a lack of ECB appear in cancellation due to lack of ECB column of time series

    # check if cancellation due to lack of ECB is flagged in patient CRF
    if row_data[42] == 1:
        
        # check if this SID appears in the cancellation due to lack of ECB field in the time series
        SID = row_data[0].replace(" ", "")
        count = 0
        for cell in time_series['otd_cancel_nobed']:
            if SID in cell:
                count += 1
                break
        if count == 0:
            errormessage(error_msg, 'The following list of patients are flagged as a cancellation due to a lack of enhanced care bed in the patient CRF but do not appear in the time series as a cancellation for this reason on any day, please check this and amend as necessary.', SID)
         
    # check if there is a reason given for a difference between predicted and actual post-op destination
    
    # check if there is a difference between predicted and actual
    if row_data[-6] != row_data[-5]:
        SID = row_data[0]
        
        # check if a reason is given or they were cancelled
        if row_data[-4] != 1 and row_data[-3] != 1 and row_data[-1] != 1 and row_data[31] != 1:
            errormessage(error_msg, 'The following list of patients were admitted to a different level of critical care than was predicted but there is no reason listed, either this was due to a lack of enhanced care bed OR this was for another reason, please flag one of these parameters for each.', SID)

In [13]:
print('If there are lots of errors thrown up, the first thing to check is whether the SIDs in the patient CRF match the hospital number + SID pairings in the EhC referrals tab of the time series. If these are not matched up then our checks will throw up lots of errors.')
print('\n')
print('Please note that when correcting issues in the Time-Series this may alter the order of anonymisation and therefore the adjacent SIDs. The CRF auto-anonymises based on the first instance of a hospital ID when reading from left to right, line by line. If a patient is added to, for example, the cancellations column, where previously the first instance was in the referrals for level 1, the SID number will decrease by however many hospital numbers it appears before in its new position. We try our best to do most of the data cleaning ourselves if we can and only if this is not possible (perhaps because it requires details we would not be able to access) do we send feedback. The patient CRF can be edited without concern for altering the SID numbers. At the end of any editing process, just before trasnfer, please perform a final check to ensure the SIDs match the correct hospital numbers or the submitted data will continue to throw up errors.')
print('\n')

for key, value in error_msg.items():
    print(key)
    print('\n')
    print(value)
    print('\n')

If there are lots of errors thrown up, the first thing to check is whether the SIDs in the patient CRF match the hospital number + SID pairings in the EhC referrals tab of the time series. If these are not matched up then our checks will throw up lots of errors.


Please note that when correcting issues in the Time-Series this may alter the order of anonymisation and therefore the adjacent SIDs. The CRF auto-anonymises based on the first instance of a hospital ID when reading from left to right, line by line. If a patient is added to, for example, the cancellations column, where previously the first instance was in the referrals for level 1, the SID number will decrease by however many hospital numbers it appears before in its new position. We try our best to do most of the data cleaning ourselves if we can and only if this is not possible (perhaps because it requires details we would not be able to access) do we send feedback. The patient CRF can be edited without concern for altering